In [4]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# GNN - Data for Node Classification Model

Create Stellargraph objects for supervised GNN.  This is much like script 40, but I reproduce the code in 2 scrips as I may want to change things.

*This script takes about 15 minutes on my MacBook Air*

In [5]:
import pandas as pd
import numpy as np

In [6]:
from pathlib import Path
import importlib
import pickle, re

In [7]:
from sba_gnn.sba_gnn import sg_gnn

In [5]:
import stellargraph as sg
from stellargraph import StellarGraph

2024-02-28 06:36:04.658863: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-28 06:36:04.658890: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-28 06:36:04.658901: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-28 06:36:04.659092: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-28 06:36:04.659112: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Import Data

In [8]:
## NN scaled input data
comb_df = pd.read_parquet(Path(setup.temp_path).joinpath('10_DATA_combined_scaled_all.parquet'))

In [9]:
with open(Path(setup.temp_path).joinpath('10_DATA_features.pkl'), 'rb') as fin:
    imputed_features = pickle.load(fin)

In [10]:
num_feat =  [c for c in imputed_features if 'NAICS' not in c]
print(num_feat)

['NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business', 'urban_flag', 'franchise_flag', 'missingindicator_LowDoc', 'missingindicator_new_business', 'missingindicator_urban_flag']


## Create Edge Map

I will have edges between a business and its NAICS, as well as edges to same-sector NAICS nodes.  This section creates general edge data, which may be filtered to create graphs

##### Create NAICS map to same-sector NAICS

In [9]:
sector_map = comb_df[['NAICS_orig', 'NAICS_sector']].drop_duplicates()

In [10]:
naics_all_sectors = sector_map.merge(sector_map.rename(columns={'NAICS_orig':'NAICS_sim'}),
                                     how='left', on ='NAICS_sector')
naics_all_sectors = naics_all_sectors[naics_all_sectors['NAICS_orig'] != naics_all_sectors['NAICS_sim']]

In [11]:
naics_all_sectors.sample(5)

,NAICS_orig,NAICS_sector,NAICS_sim
66616,454111,44-45,451110
260508,322110,31-33,333312
152599,322130,31-33,336330
5256,339116,31-33,334419
195848,327310,31-33,339993


##### Get edges from businesses to their exact NAICS

In [12]:
# Edges from businesses to their NAICS code
naics_info_df = comb_df[['LoanNr_ChkDgt', 'NAICS_orig', 'dset']].copy() \
    .rename(columns={'LoanNr_ChkDgt':'source'}) 
naics_info_df['target'] = sg_gnn.get_naics_index(naics_info_df['NAICS_orig'])

edges_business_naics = naics_info_df[['source', 'target', 'dset']].copy()
edges_business_naics['type'] = 'loan_naics'

In [13]:
naics_info_df.head(3)

,source,NAICS_orig,dset,target
index,,,,
1,1000024006,722410,test,n_722410
8,1000146010,445299,test,n_445299
27,1000704009,722410,test,n_722410


In [14]:
edges_businesses_naics_sim = naics_info_df.drop(columns='target', errors='ignore') \
    .merge(naics_all_sectors, how='inner', on='NAICS_orig')
edges_businesses_naics_sim['target'] = sg_gnn.get_naics_index(edges_businesses_naics_sim['NAICS_sim'])
edges_businesses_naics_sim = edges_businesses_naics_sim[['source', 'target', 'dset']]
edges_businesses_naics_sim['type'] = 'loan_sector_naics'
print(f'Sector NAICS edges: {edges_businesses_naics_sim.shape}')

Sector NAICS edges: (67912366, 4)


In [15]:
edges_all = pd.concat([edges_business_naics, edges_businesses_naics_sim], axis=0) \
    .reset_index(drop=True)

In [16]:
edges_all['type'].value_counts()

type
loan_sector_naics    67912366
loan_naics             688081
Name: count, dtype: int64

In [17]:
print(edges_all[['source', 'target']].drop_duplicates().shape)
print(edges_all.shape)

(68600447, 2)
(68600447, 4)


In [18]:
edges_all.to_parquet(Path(setup.temp_path).joinpath('20_DATA_graph_edges.parquet'))

## Get Node Features
Use the scaled NN data.  For NAICS, just set an indicator for now.  

In [19]:
features_business = comb_df[['LoanNr_ChkDgt', 'dset', 'NAICS_orig'] + num_feat] \
    .set_index('LoanNr_ChkDgt')
features_business.to_parquet(Path(setup.temp_path).joinpath('20_DATA_graph_node_features_loans.parquet'))

In [20]:
# NAICS features - Just use indicator.  Include original code for mapping
features_naics = comb_df[['NAICS_orig']].drop_duplicates()
features_naics['feat'] = 1
features_naics['source'] = sg_gnn.get_naics_index(features_naics['NAICS_orig'])
features_naics.set_index('source', inplace=True) 
print(features_naics.shape)
features_naics.to_parquet(Path(setup.temp_path).joinpath('20_DATA_graph_node_features_naics.parquet'))

(1311, 2)


## Stellargraph - train and validation
Graph without test nodes (make sure set-aside NAICS not included)

In [21]:
# Get only relevant edges and node data
edges_train_val, features_business_train_val, features_naics_train_val = \
    sg_gnn.limit_data(edges_all, features_business, features_naics.drop(columns='NAICS_orig'),
                     ['train', 'val'])

In [22]:
print(f'Train+val graph data')
print(f'business features, start rows {features_business.shape[0]}, end {features_business_train_val.shape[0]}')
print(f'naics features, start rows {features_naics.shape[0]}, end {features_naics_train_val.shape[0]}')
print(f'edges, start rows {edges_all.shape[0]}, end {edges_train_val.shape[0]}')

Train+val graph data
business features, start rows 688081, end 551635
naics features, start rows 1311, end 1240
edges, start rows 68600447, end 52159047


In [23]:
sba_graph_train_val = StellarGraph({'LoanNr_ChkDgt':features_business_train_val.drop(columns='NAICS_orig'),
                                    'NAICS': features_naics_train_val},
                                   edges_train_val, 
                                   source_column="source", target_column="target",
                                   edge_type_column="type")

In [24]:
print(sba_graph_train_val.info())

StellarGraph: Undirected multigraph
 Nodes: 552875, Edges: 52159047

 Node types:
  LoanNr_ChkDgt: [551635]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS, LoanNr_ChkDgt-loan_sector_naics->NAICS
  NAICS: [1240]
    Features: float32 vector, length 1
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-loan_sector_naics->LoanNr_ChkDgt

 Edge types:
    LoanNr_ChkDgt-loan_sector_naics->NAICS: [51607412]
        Weights: all 1 (default)
        Features: none
    LoanNr_ChkDgt-loan_naics->NAICS: [551635]
        Weights: all 1 (default)
        Features: none


In [25]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('20_DATA_stellargraph_train_val.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_train_val, fout)

## Stellargraph - all nodes

In [26]:
sba_graph_all = StellarGraph({'LoanNr_ChkDgt':features_business.drop(columns=['dset', 'NAICS_orig']) ,
                              'NAICS': features_naics.drop(columns='NAICS_orig')},
                             edges_all.drop(columns=['dset']).drop_duplicates(),
                             source_column="source", target_column="target",
                             edge_type_column="type")

In [27]:
print(sba_graph_all.info())

StellarGraph: Undirected multigraph
 Nodes: 689392, Edges: 68600447

 Node types:
  LoanNr_ChkDgt: [688081]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS, LoanNr_ChkDgt-loan_sector_naics->NAICS
  NAICS: [1311]
    Features: float32 vector, length 1
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-loan_sector_naics->LoanNr_ChkDgt

 Edge types:
    LoanNr_ChkDgt-loan_sector_naics->NAICS: [67912366]
        Weights: all 1 (default)
        Features: none
    LoanNr_ChkDgt-loan_naics->NAICS: [688081]
        Weights: all 1 (default)
        Features: none


In [28]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('20_DATA_stellargraph_all.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_all, fout)

## Label Data

In [11]:
label_df = comb_df[['LoanNr_ChkDgt', 'target', 'dset', 'dset_naics_holdout']] \
    .set_index('LoanNr_ChkDgt').sort_index()

In [13]:
label_df.to_parquet(Path(setup.temp_path).joinpath('20_DATA_label_info.parquet'))